In [3]:
import os
import json

In [4]:
train_dicts = [json.loads(item) for item in open('../data/FEVER/train.jsonl', encoding='utf8').readlines()]
dev_dicts = [json.loads(item) for item in open('../data/FEVER/paper_dev.jsonl', encoding='utf8').readlines()]

In [29]:
def clean_line(line):
    line = line[2:line[2:].find('\t')]
    line = line.replace('-LRB-', '(')
    line = line.replace('-RRB-', ')')
    line = line.replace('-LSB-', '[')
    line = line.replace('-RSB-', ']')
    line = line.replace('( ', '(')
    line = line.replace(' )', ')')
    line = line.replace('[ ', '[')
    line = line.replace(' ]', ']')
    line = line.replace(' .', '.')
    line = line.replace(' , ', ', ')
    line = line.replace(' ; ', '; ')
    line = line.replace(' : ', ': ')
    return line

In [30]:
wiki_lines_dict = {}
_wiki_folder = '../data/FEVER/wiki-pages'
for file in os.listdir(_wiki_folder):
    for item in open(f'{_wiki_folder}/{file}', encoding='utf8').readlines():
        wiki_dict = json.loads(item)
        wiki_id = wiki_dict['id']
        lines = [clean_line(line) for line in wiki_dict['lines'].split('\n')]
        wiki_lines_dict[wiki_id] = lines

In [31]:
def populate_texts_for_label(data_dicts, label):
    all_texts = []

    for item in data_dicts:
        if item['label'] == label:
            text = ''
            for paragraph in item['evidence']:
                for line in paragraph:
                    wiki_id = line[-2]
                    sentence_num = line[-1]
                    if wiki_id not in wiki_lines_dict:
                        continue
                    text += wiki_lines_dict[wiki_id][sentence_num] + '. '
                if text:
                    text += '\n'
            text = text.strip()
            if text and text != '.':
                claim = item['claim']
                all_texts.append('Evidence:\n'
                                 + text + '\n\n'
                                 + 'Claim:\n'
                                 + claim
                                )
    return all_texts

In [32]:
supporting_texts = populate_texts_for_label(train_dicts, 'SUPPORTS')
refuting_texts = populate_texts_for_label(train_dicts, 'REFUTES')

In [33]:
len(supporting_texts)

69027

In [34]:
len(refuting_texts)

26212

In [35]:
json.dump(supporting_texts, open('../data/supporting.json', 'w'))
json.dump(refuting_texts, open('../data/refuting.json', 'w'))